In [1]:
from slt_positional_bias.dataset import generate_data_frame, generate_merged_data_frame

df_merged = generate_merged_data_frame()
df_merged


2025-07-09 21:00:44.075 | INFO     | slt_positional_bias.config:<module>:11 - PROJ_ROOT path is: C:\Users\marti\slt_group_2_positional_bias


2025-07-09 21:00:45.108 | INFO     | slt_positional_bias.dataset:generate_data_frame:48 - file path exists
2025-07-09 21:00:51.084 | INFO     | slt_positional_bias.dataset:generate_data_frame:48 - file path exists
2025-07-09 21:00:51.089 | INFO     | slt_positional_bias.dataset:generate_data_frame:48 - file path exists


,qid,doc_id,rel,text_x,text_y
0,2024-145979,msmarco_v2.1_doc_01_523681915#0_449763684,2,what is vicarious trauma and how can it be cop...,Vicarious traumatization - Wikipedia Vicarious...
1,2024-145979,msmarco_v2.1_doc_13_1647729865#0_3617397938,3,what is vicarious trauma and how can it be cop...,Vicarious Trauma: What Causes it and Ways to H...
2,2024-145979,msmarco_v2.1_doc_25_1148700328#0_2179512504,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
3,2024-145979,msmarco_v2.1_doc_25_1148700328#1_2179514079,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
4,2024-145979,msmarco_v2.1_doc_44_1043805224#2_2182641075,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? - Jefferson Center -...
...,...,...,...,...,...
108473,2024-21669,msmarco_v2.1_doc_08_298503251#1_568833050,0,NaN,NaN
108474,2024-21669,msmarco_v2.1_doc_27_877180220#2_1765802089,1,NaN,NaN
108475,2024-21669,msmarco_v2.1_doc_01_1752638978#7_2584141250,1,NaN,NaN
108476,2024-21669,msmarco_v2.1_doc_09_914336366#11_1554208777,0,NaN,NaN


In [ ]:
API_URL = "https://api.helmholtz-blablador.fz-juelich.de/v1/"
API_KEY = "key"
API_MODEL = "alias-fast"

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=API_URL
)

In [6]:
import pandas as pd

# Step 1: Prepare data
first_qid = df_merged['qid'].iloc[0]
df_first_qid = df_merged[df_merged['qid'] == first_qid]

# One relevant and 19 irrelevant docs
rel_3_doc = df_first_qid[df_first_qid['rel'] == 3].head(1)
rel_0_docs = df_first_qid[df_first_qid['rel'] == 0].head(19)

rel_3_text = rel_3_doc['text_y'].iloc[0]
query = rel_3_doc['text_x'].iloc[0]
rel_0_texts = rel_0_docs['text_y'].dropna().tolist()

# Positions where to insert rel=3 (1-based for readability)
positions = [0, 4, 9, 14, 19]

# Step 2–4: Loop through positions, send to model, evaluate
for idx, pos in enumerate(positions, 1):
    # Construct context with rel=3 at the specified position
    text_list = rel_0_texts.copy()
    if pos >= len(text_list):
        text_list.append(rel_3_text)
    else:
        text_list.insert(pos, rel_3_text)
    context_string = ' '.join(text_list)

    # Construct user prompt
    user_prompt = f"""
    Context:
    {context_string}

    Question:
    {query}

    Answer:"""

    # System prompt
    system_prompt = "You are a helpful assistant answering a question based on retrieved context information."

    # Ask the model
    response = client.chat.completions.create(
        model="alias-fast",
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    answer = response.choices[0].message.content.strip()

    # Evaluation prompt
    evaluation_prompt = f"""
    The following context was given to a language model. One section (rel=3) was highly relevant, and 19 others (rel=0) were not.

    Here is the relevant section (rel=3):
    \"\"\"
    {rel_3_text}
    \"\"\"

    Here is the model's answer to a question based on all 20 sections:
    \"\"\"
    {answer}
    \"\"\"

    Question: Is the answer **mainly based** on the relevant section (rel=3)?

    Respond with only "Yes" or "No".
    """

    # Evaluate
    eval_response = client.chat.completions.create(
        model="alias-fast",
        temperature=0,
        messages=[
            {"role": "user", "content": evaluation_prompt}
        ],
    )
    eval_result = eval_response.choices[0].message.content.strip()

    # Print result
    print(f"\n--- Run {idx} (rel=3 at position {pos+1}) ---")

    print("Is answer based on rel=3?:", eval_result)


--- Run 1 (rel=3 at position 1) ---
Is answer based on rel=3?: Yes

--- Run 2 (rel=3 at position 5) ---
Is answer based on rel=3?: No

--- Run 3 (rel=3 at position 10) ---
Is answer based on rel=3?: Yes

--- Run 4 (rel=3 at position 15) ---
Is answer based on rel=3?: Yes

--- Run 5 (rel=3 at position 20) ---
Is answer based on rel=3?: Yes
